#**Import Modules**

Selenium, Openpyxl, Files, Time

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 2s (141 kB/s)
Reading package lists... Done
Reading package lists... Done
B

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *

In [0]:
import time

In [0]:
import openpyxl

In [0]:
from google.colab import files

#**Fetch all Volume Links**

In [0]:
VOLURLFORMAT = "https://onlinelibrary.wiley.com/loi/14679701/year/"
volUrls = []

for yr in range(1977, 2021):
    wd.get(VOLURLFORMAT + str(yr))
    time.sleep(0.5)

    issue_tags = wd.find_elements_by_xpath("//div[@class='loi__issue']/h4/a[@href]")
    for issue in issue_tags:
        volUrls.append([yr, issue.get_attribute("href")])
        
volUrls

[[1977, 'https://onlinelibrary.wiley.com/toc/14679701/1977/1/1'],
 [1978, 'https://onlinelibrary.wiley.com/toc/14679701/1978/1/4'],
 [1978, 'https://onlinelibrary.wiley.com/toc/14679701/1978/1/3'],
 [1978, 'https://onlinelibrary.wiley.com/toc/14679701/1978/1/2'],
 [1979, 'https://onlinelibrary.wiley.com/toc/14679701/1979/2/3'],
 [1979, 'https://onlinelibrary.wiley.com/toc/14679701/1979/2/2'],
 [1979, 'https://onlinelibrary.wiley.com/toc/14679701/1979/2/1'],
 [1980, 'https://onlinelibrary.wiley.com/toc/14679701/1980/3/3'],
 [1980, 'https://onlinelibrary.wiley.com/toc/14679701/1980/3/2'],
 [1980, 'https://onlinelibrary.wiley.com/toc/14679701/1980/3/1'],
 [1980, 'https://onlinelibrary.wiley.com/toc/14679701/1980/2/4'],
 [1981, 'https://onlinelibrary.wiley.com/toc/14679701/1981/4/4'],
 [1981, 'https://onlinelibrary.wiley.com/toc/14679701/1981/4/2%E2%80%903'],
 [1981, 'https://onlinelibrary.wiley.com/toc/14679701/1981/4/1'],
 [1981, 'https://onlinelibrary.wiley.com/toc/14679701/1981/3/4'],


#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
volUrls.reverse()

In [0]:
r = 2

# write header
ws.cell(row = 1, column = 1).value = 'journal'
ws.cell(row = 1, column = 2).value = 'year'
ws.cell(row = 1, column = 3).value = 'title'
ws.cell(row = 1, column = 4).value = 'keywords'
ws.cell(row = 1, column = 5).value = 'authors'

for i in range(len(volUrls)):
    # access volume page
    print("Reading volume, " + volUrls[i][1])
    wd.get(volUrls[i][1])
    time.sleep(1)

    # fetch year
    yr = volUrls[i][0]

    # fetch article urls
    articleUrls = []
    a_tags = wd.find_elements_by_xpath("//a[@class='issue-item__title visitable'][@href]")
    for tag in a_tags:
        if (not "Issue Information" in tag.text) and (not tag.get_attribute("href") in articleUrls):
            articleUrls.append(tag.get_attribute("href"))
    
    for articleurl in articleUrls:
        # access article page
        wd.get(articleurl)
        time.sleep(0.8)
        #print("Reading article, " + articleurl)

        # fetch authors
        authors = []
        auth_tags = wd.find_elements_by_xpath("//a[@class='author-name accordion-tabbed__control']/span")
        for auth in auth_tags:
            if not auth.get_attribute("innerText") in authors:
                authors.append(auth.get_attribute("innerText"))
        if(len(authors) == 0):
            continue
        #print(authors)

        # fetch title
        try:
            title = wd.find_element_by_xpath("//h1[@class='citation__title']").text
        except NoSuchElementException:
            continue
        #print(title)

        # fetch keywords
        keywords = []
        kwd_tags = wd.find_elements_by_xpath("//ul[@class='rlist rlist--inline']/li")
        for kwd in kwd_tags:
            keywords.append(kwd.get_attribute("innerText"))
        if(len(keywords) == 0):
            continue
        #print(keywords)

        # write data to excel file
        print("Writing, " + title)
        ws.cell(row = r, column = 1).value = "The World Economy"
        ws.cell(row = r, column = 2).value = yr
        ws.cell(row = r, column = 3).value = title
        for j in range(len(keywords)):
            ws.cell(row = r + j, column = 4).value = keywords[j]
        for j in range(len(authors)):
            ws.cell(row = r + j, column = 5).value = authors[j]
        r += max([len(keywords), len(authors)])

    # backup for every ten volumes
    if(i % 20 == 0):
        print("Saving backup file")
        wb.save(filename = "the_world_economy_backup.xlsx")
        time.sleep(20)
        #files.download("journal_of_international_economics_backup.xlsx")

Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1977/1/1
Saving backup file
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1978/1/4
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1978/1/3
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1978/1/2
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1979/2/3
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1979/2/2
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1979/2/1
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1980/3/3
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1980/3/2
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1980/3/1
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1980/2/4
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1981/4/4
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/1981/4/2%E2%80%903
Reading volume, https://onlinelibrary.wiley.com/toc/14679701/

#**Save File**

In [0]:
wb.save(filename = "the_world_economy.xlsx")

#**Download File**

In [0]:
files.download("the_world_economy.xlsx")